# Dikti Scraper 0.1

## melakukan pengambilan data unstructure dari halaman web

##### membuat objek scrape

In [ ]:
from bs4 import BeautifulSoup
import urllib
import pandas
import MySQLdb
import os


db = MySQLdb.connect(
    host = '127.0.0.1',
    user = 'root',
    passwd = '310116',
    db = 'diktiScraperdb'
)
print "Databases Connected"
class scrape(object): #membuat objek scrape

    def __init__(self, nameOfMachine): #inisiasi objek
        self.nameOfMachine = nameOfMachine #pemberian atribut objek pada inisiasi

    def getLink(self, addr): #membuat fungsi getLink
        dataLink = {  #membuat dictionary
        'addt' : [], #teks untuk penambahan
        'link' : [] #link yang ada di halaman
        }
        def scrapePage(addr):
            scrapeData = BeautifulSoup(urllib.urlopen(addr), "html.parser")
            return scrapeData
        data = scrapePage(addr)
        for recordRow in data.findAll('tr'):
            addtText = ""
            for recordLink in recordRow.findAll('a', href=True):
                tempData = recordLink.text.split()
                for listOfTempData in tempData:
                    addtText = addtText + listOfTempData + " "
                dataLink['addt'].append(str(addtText))
                dataLink['link'].append(str(recordLink['href']))
        print "Get DataLink Scrape HTML Object"        
        return dataLink
    def getTextOnly(self,addr):
        dataText = {
            'addt' : []
        }
        listText = []
        def scrapePage(addr):
            scrapeData = BeautifulSoup(urllib.urlopen(addr), "html.parser")
            return scrapeData
        data = scrapePage(addr)
        for recordRow in data.findAll('tr'):
            for recordText in recordRow.findAll('td'):
                listText.append(recordText.text)
        for item in listText[2::21]:
            addtText = ""
            tempData = item.split()
            for listOfTempData in tempData:
                addtText = addtText + listOfTempData + " "
            dataText['addt'].append(str(addtText))
            #print addtText
        print "Get Text Faculty ScrapeHTML Object"
        return dataText


Databases Connected


In [ ]:
urlServer = "/home/alien/diktiScraper" #server program host
urlTarget = "http://forlap.dikti.go.id/perguruantinggi/homerekap" #home rekap
dataGet = {
    'addt' : [],
    'link' : []
} #dictionary for data that got from scraper
scraperLayer0 = scrape('layer0_df')
dataGet = scraperLayer0.getLink(urlTarget)
layer0_df = pandas.DataFrame(dataGet, columns=['addt', 'link'])
layer0_df.index.name = 'institutionId'
#print layer0_df
cur = db.cursor()
cur.execute('USE diktiScraperdb;') #menggunakan db
cur.execute('DROP TABLE IF EXISTS institusi;') #mengecek tablenya sudah ada atau belum
cur.execute('CREATE TABLE institusi (institutionId INT NOT NULL PRIMARY KEY, addt VARCHAR(255),link VARCHAR(255));') #buat table
layer0_df.to_sql(con=db,
                 name='institusi', 
                 if_exists='append', 
                 flavor='mysql',              
                ) #mencetak ke mySQL
print "Data Layer 0 transfered to SQL, done"
layer0_df = pandas.read_sql("SELECT link FROM institusi;" ,db)
print "Data load from SQL layer0_df"
i = 0
#layer = 1
for link in layer0_df['link']:
    urlTarget0 = link
    #print urlTarget
    if i/10 < 1:
        nameDb1 = 'list0' + str(i)
    else :
        nameDb1 = 'list' + str(i)
    nameDb1 = str(nameDb1)
    query0 = 'USE diktiScraperdb;'
    query1 = "DROP TABLE IF EXISTS " + nameDb1
    query2 = "CREATE TABLE "+ nameDb1 +" (institutionId INT NOT NULL PRIMARY KEY, addt VARCHAR(255),link VARCHAR(255));"
    scraper1 = scrape('df1')
    dataGet1 = scraper1.getLink(urlTarget0)
    df1 = pandas.DataFrame(dataGet1, columns=['addt', 'link'])
    df1.index.name = 'institutionId'
    #print df
    cur.execute(query0)
    cur.execute(query1)
    cur.execute(query2)
    df1.to_sql(con=db,
            name= nameDb1, 
            if_exists='append', 
            flavor='mysql',
            ) #mencetak ke mySQL
    print nameDb1 +" transfered to SQL, done"
    query3 = "SELECT link FROM "+ nameDb1 +";"
    layer1_df = pandas.read_sql(query3 ,db)
    print "Data load from SQL " + nameDb1
    j = 0
    for link1 in  layer1_df['link']:
        urlTarget1 = link1
        if j/10 < 1:
            nameDb2 = nameDb1 + "0" + str(j)
        else :
            nameDb2 = nameDb1 + str(j)
        nameDb2 = str(nameDb2)
        query4 = "DROP TABLE IF EXISTS " + nameDb2
        query5 = "CREATE TABLE "+ nameDb2 +" (institutionId INT NOT NULL PRIMARY KEY, addt VARCHAR(255),link VARCHAR(255));"
        scraper2 = scrape('df2')
        dataGet2 = scraper2.getLink(urlTarget1)
        df2 = pandas.DataFrame(dataGet2, columns=['addt', 'link'])
        df2.index.name = 'institutionId'
        cur.execute(query0)
        cur.execute(query4)
        cur.execute(query5)
        df2.to_sql(con=db,
                  name = nameDb2,
                  if_exists="append",
                  flavor="mysql",
                  )
        print nameDb2 +" transfered to SQL, done"
        query6 = "SELECT link FROM "+ nameDb2 +";"
        layer2_df = pandas.read_sql(query6 ,db)
        print "Data load from SQL " + nameDb2
        k = 0
        for link2 in  layer2_df['link']:
            urlTarget2 = link2
            if k/10 < 1:
                nameDb3 = nameDb2 + "0" + str(k)
            else :
                nameDb3 = nameDb2 + str(k)
            nameDb3 = str(nameDb3)
            query7 = "DROP TABLE IF EXISTS " + nameDb3 +";"
            query8 = "CREATE TABLE "+ nameDb3 +" (institutionId INT NOT NULL PRIMARY KEY, addt VARCHAR(255));"
            scraper3 = scrape('df3')
            dataGet3 = scraper3.getTextOnly(urlTarget2)
            df3 = pandas.DataFrame(dataGet3, columns=['addt'])
            df3.index.name = 'institutionId'
            cur.execute(query0)
            cur.execute(query7)
            cur.execute(query8)
            df3.to_sql(con=db,
                      name = nameDb3,
                      if_exists="append",
                      flavor="mysql",
                      )
            print nameDb3 +" transfered to SQL, done"
            k = k+1
        j = j+1
    i = i+1

Get DataLink Scrape HTML Object
Data Layer 0 transfered to SQL, done
Data load from SQL layer0_df
Get DataLink Scrape HTML Object
list00 transfered to SQL, done
Data load from SQL list00
Get DataLink Scrape HTML Object
list0000 transfered to SQL, done
Data load from SQL list0000
Get DataLink Scrape HTML Object
list0001 transfered to SQL, done
Data load from SQL list0001
Get DataLink Scrape HTML Object
list0002 transfered to SQL, done
Data load from SQL list0002
Get DataLink Scrape HTML Object
list0003 transfered to SQL, done
Data load from SQL list0003
Get DataLink Scrape HTML Object
list0004 transfered to SQL, done
Data load from SQL list0004
Get DataLink Scrape HTML Object
list0005 transfered to SQL, done
Data load from SQL list0005
Get DataLink Scrape HTML Object
list0006 transfered to SQL, done
Data load from SQL list0006
Get DataLink Scrape HTML Object
list0007 transfered to SQL, done
Data load from SQL list0007
Get DataLink Scrape HTML Object
list0008 transfered to SQL, done
Data

/home/alien/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:95: Warning: Unknown table 'diktiScraperdb.list010100'


Get Text Faculty ScrapeHTML Object
list010101 transfered to SQL, done


/home/alien/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:95: Warning: Unknown table 'diktiScraperdb.list010101'


Get Text Faculty ScrapeHTML Object
list010102 transfered to SQL, done


...

/home/alien/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:95: Warning: Unknown table 'diktiScraperdb.list010401'


Get Text Faculty ScrapeHTML Object
list010402 transfered to SQL, done


/home/alien/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:95: Warning: Unknown table 'diktiScraperdb.list010402'


Get Text Faculty ScrapeHTML Object
list010403 transfered to SQL, done


/home/alien/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:95: Warning: Unknown table 'diktiScraperdb.list010403'
